Universal imports

In [1]:
import numpy as np
import pygame
import time
import os
import matplotlib.pyplot as plt
from ball_simulation import Ball
from environment import BallEnv
import config
import importlib

<frozen importlib._bootstrap>:241: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.6.1 (SDL 2.32.56, Python 3.10.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


Configuration parameters can be modified in config.py

P-Controller

In [ ]:
from p_controller import PController
importlib.reload(config)

def run_p_controller_sim():
    """Runs the simulation with the P-Controller."""
    pygame.init()
    screen = pygame.display.set_mode((config.SCREEN_WIDTH, config.SCREEN_HEIGHT))
    pygame.display.set_caption("Ball Simulator - P Controller")
    clock = pygame.time.Clock()
    font = pygame.font.Font(None, 30)

    ball = Ball(config.SCREEN_WIDTH / 2, config.GROUND_HEIGHT + config.BALL_RADIUS)
    p_controller = PController(kp=0.8) # You can tune this Kp value

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        force = p_controller.get_action(ball.y, config.TARGET_HEIGHT, ball.velocity)
        ball.apply_force(force)

        # --- Drawing ---
        screen.fill(config.WHITE)
        pygame.draw.line(screen, config.BLACK, (0, config.SCREEN_HEIGHT - config.GROUND_HEIGHT), (config.SCREEN_WIDTH, config.SCREEN_HEIGHT - config.GROUND_HEIGHT), 2)
        pygame.draw.line(screen, config.GREEN, (0, config.SCREEN_HEIGHT - config.TARGET_HEIGHT), (config.SCREEN_WIDTH, config.SCREEN_HEIGHT - config.TARGET_HEIGHT), 2)
        target_text = font.render('Target Height', True, config.GREEN)
        screen.blit(target_text, (5, config.SCREEN_HEIGHT - config.TARGET_HEIGHT - 25))
        ball.draw(screen)

        # --- Info Text ---
        height_text = font.render(f'Height: {ball.y:.2f}', True, config.BLACK)
        velocity_text = font.render(f'Velocity: {ball.velocity:.2f}', True, config.BLACK)
        force_text = font.render(f'Force: {force:.2f}', True, config.BLACK)
        screen.blit(height_text, (10, 10))
        screen.blit(velocity_text, (10, 40))
        screen.blit(force_text, (10, 70))

        pygame.display.flip()
        clock.tick(1 / config.TIME_STEP)

    pygame.quit()
    
env = BallEnv()
run_p_controller_sim()

Standard MPC Controller

In [ ]:
from mpc_controller import MPCController
from filter import *
importlib.reload(config)

def run_mpc_controller_sim(estimator: int):
    """Runs the simulation with the MPC Controller."""
    pygame.init()
    screen = pygame.display.set_mode((config.SCREEN_WIDTH, config.SCREEN_HEIGHT))
    pygame.display.set_caption("Ball Simulator - MPC Controller")
    clock = pygame.time.Clock()
    font = pygame.font.Font(None, 30)


    ball = Ball(config.SCREEN_WIDTH / 2, config.GROUND_HEIGHT + config.BALL_RADIUS)
    mpc_controller = MPCController(N=config.STD_MPC_HORIZON, dt=config.TIME_STEP)

    # --- Initialize chosen estimator ---
    if estimator == 2:
        ekf = EKF(
            dynamic_model(config.EKF_VAR_PROC_POS, config.EKF_VAR_PROC_VEL),
            sensor_model(config.EKF_VAR_MEAS_POS, config.EKF_VAR_MEAS_VEL)
        )
    if estimator == 3:
        mhe = MHE(
            dynamic_model(config.MHE_VAR_PROC_POS, config.MHE_VAR_PROC_VEL),
            sensor_model(config.MHE_VAR_MEAS_POS, config.MHE_VAR_MEAS_VEL),
            config.MHE_HORIZON
        )
    
    positions = []
    velocities = []
    forces = []
    predicted_trajectories = []
    predicted_controls = []

    measurements = np.empty((2,0))

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False


        # --- State estimation ---
        z_meas = add_noise(ball.y, ball.velocity)
        measurements = np.append(measurements, z_meas, axis=1)

        if estimator == 2: # EKF
            est_pos, est_vel = run_ekf(ekf, z_meas, forces)
        elif estimator == 3: # MHE
            est_pos, est_vel = run_mhe(mhe, z_meas, forces)
        else: # no estimator, use ground truth
            est_pos, est_vel = ball.y, ball.velocity
        

        force, pred_X, pred_U = mpc_controller.get_action(est_pos, est_vel)
        # apply first control
        ball.apply_force(force, disturbance=False)

        positions.append(ball.y)
        velocities.append(ball.velocity)
        forces.append(force)

        # store predicted trajectory (convert to simple lists) if available
        if pred_X is not None:
            # pred_X shape (2, N+1) -> store heights and velocities separately or together
            predicted_trajectories.append(pred_X.tolist())
        else:
            predicted_trajectories.append(None)

        if pred_U is not None:
            predicted_controls.append(pred_U.flatten().tolist())
        else:
            predicted_controls.append(None)


        # --- Drawing ---
        screen.fill(config.WHITE)
        pygame.draw.line(screen, config.BLACK, (0, config.SCREEN_HEIGHT - config.GROUND_HEIGHT), (config.SCREEN_WIDTH, config.SCREEN_HEIGHT - config.GROUND_HEIGHT), 2)
        pygame.draw.line(screen, config.GREEN, (0, config.SCREEN_HEIGHT - config.TARGET_HEIGHT), (config.SCREEN_WIDTH, config.SCREEN_HEIGHT - config.TARGET_HEIGHT), 2)
        target_text = font.render('Target Height', True, config.GREEN)
        screen.blit(target_text, (5, config.SCREEN_HEIGHT - config.TARGET_HEIGHT - 25))
        ball.draw(screen)

        # --- Info Text ---
        height_text = font.render(f'Height: {ball.y:.2f}', True, config.BLACK)
        velocity_text = font.render(f'Velocity: {ball.velocity:.2f}', True, config.BLACK)
        force_text = font.render(f'Force: {force:.2f}', True, config.BLACK)
        screen.blit(height_text, (10, 10))
        screen.blit(velocity_text, (10, 40))
        screen.blit(force_text, (10, 70))

        pygame.display.flip()
        clock.tick(1 / config.TIME_STEP)

    pygame.quit()
    qx, qu, lbu, ubu, r, delta_u_max = mpc_controller.sizes()
    ref = config.TARGET_HEIGHT
    np.savez("mpc_data.npz", positions=positions, forces=forces, trajectories=predicted_trajectories, controls=predicted_controls, N=config.STD_MPC_HORIZON, qx=qx, qu=qu, lbu=lbu, ubu=ubu, r=r, ref=ref, delta_u_max=delta_u_max)

    if estimator == 2:
        np.savez("ekf_data.npz", ground_truth=[positions, velocities],
                measurements=measurements,
                estimated_states=ekf.state_ests,
                estimated_measurements=ekf.meas_ests)
    if estimator == 3:
        np.savez("mhe_data.npz", ground_truth=[positions, velocities],
                measurements=measurements,
                estimated_states=mhe.x_ests)

env = BallEnv()
run_mpc_controller_sim(1)  # 1: No estimator, 2: EKF, 3: MHE

Stochastic MPC Controller

In [4]:
from mpc_controller_stoch import MPCControllerStochastic

def run_mpc_controller_stochastic_sim(estimator: int):
    """Runs the simulation with the stochastic MPC Controller."""
    pygame.init()
    screen = pygame.display.set_mode((config.SCREEN_WIDTH, config.SCREEN_HEIGHT))
    pygame.display.set_caption("Ball Simulator - stochastic MPC Controller")
    clock = pygame.time.Clock()
    font = pygame.font.Font(None, 30)

    ball = Ball(config.SCREEN_WIDTH / 2, config.GROUND_HEIGHT + config.BALL_RADIUS)
    mpc_controller_stoch = MPCControllerStochastic(N=config.STOCHASTIC_MPC_HORIZON, dt=config.TIME_STEP, num_samples=config.STOCHASTIC_MPC_SAMPLES) # You can tune N, dt and num_samples values

    # --- Initialize chosen estimator ---
    if estimator == 2:
        ekf = EKF(
            dynamic_model(config.EKF_VAR_PROC_POS, config.EKF_VAR_PROC_VEL),
            sensor_model(config.EKF_VAR_MEAS_POS, config.EKF_VAR_MEAS_VEL)
        )
    if estimator == 3:
        mhe = MHE(
            dynamic_model(config.MHE_VAR_PROC_POS, config.MHE_VAR_PROC_VEL),
            sensor_model(config.MHE_VAR_MEAS_POS, config.MHE_VAR_MEAS_VEL),
            config.MHE_HORIZON
        )
    
    positions = []
    velocities = []
    forces = []
    predicted_trajectories = []
    predicted_controls = []

    measurements = np.empty((2,0))

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False


        # --- State estimation ---
        z_meas = add_noise(ball.y, ball.velocity)
        measurements = np.append(measurements, z_meas, axis=1)

        if estimator == 2: # EKF
            est_pos, est_vel = run_ekf(ekf, z_meas, forces)
        elif estimator == 3: # MHE
            est_pos, est_vel = run_mhe(mhe, z_meas, forces)
        else: # no estimator, use ground truth
            est_pos, est_vel = ball.y, ball.velocity

        
        force, pred_X, pred_U = mpc_controller_stoch.get_action(est_pos, est_vel)
        # apply first control
        ball.apply_force(force, disturbance=True)

        positions.append(ball.y)
        velocities.append(ball.velocity)
        forces.append(force)

        # store predicted trajectory (convert to simple lists) if available
        if pred_X is not None:
            # pred_X shape (2, N+1) -> store heights and velocities separately or together
            predicted_trajectories.append(pred_X.tolist())
        else:
            predicted_trajectories.append(None)

        if pred_U is not None:
            predicted_controls.append(pred_U.flatten().tolist())
        else:
            predicted_controls.append(None)


        # --- Drawing ---
        screen.fill(config.WHITE)
        pygame.draw.line(screen, config.BLACK, (0, config.SCREEN_HEIGHT - config.GROUND_HEIGHT), (config.SCREEN_WIDTH, config.SCREEN_HEIGHT - config.GROUND_HEIGHT), 2)
        pygame.draw.line(screen, config.GREEN, (0, config.SCREEN_HEIGHT - config.TARGET_HEIGHT), (config.SCREEN_WIDTH, config.SCREEN_HEIGHT - config.TARGET_HEIGHT), 2)
        target_text = font.render('Target Height', True, config.GREEN)
        screen.blit(target_text, (5, config.SCREEN_HEIGHT - config.TARGET_HEIGHT - 25))
        ball.draw(screen)

        # --- Info Text ---
        height_text = font.render(f'Height: {ball.y:.2f}', True, config.BLACK)
        velocity_text = font.render(f'Velocity: {ball.velocity:.2f}', True, config.BLACK)
        force_text = font.render(f'Force: {force:.2f}', True, config.BLACK)
        screen.blit(height_text, (10, 10))
        screen.blit(velocity_text, (10, 40))
        screen.blit(force_text, (10, 70))

        pygame.display.flip()
        clock.tick(1 / config.TIME_STEP)

    pygame.quit()
    qx, qu, lbu, ubu, r, delta_u_max = mpc_controller_stoch.sizes()
    ref = config.TARGET_HEIGHT
    np.savez("mpc_data.npz", positions=positions, forces=forces, trajectories=predicted_trajectories, controls=predicted_controls, N=config.STOCHASTIC_MPC_HORIZON, qx=qx, qu=qu, lbu=lbu, ubu=ubu, r=r, ref=ref, delta_u_max=delta_u_max)

    if estimator == 2:
        np.savez("ekf_data.npz", ground_truth=[positions, velocities],
                measurements=measurements,
                estimated_states=ekf.state_ests,
                estimated_measurements=ekf.meas_ests)
    if estimator == 3:
        np.savez("mhe_data.npz", ground_truth=[positions, velocities],
                measurements=measurements,
                estimated_states=mhe.x_ests)
        
env = BallEnv()
run_mpc_controller_stochastic_sim(1)  # 1: No estimator, 2: EKF, 3: MHE

c:\Users\mariujf\anaconda3\envs\RLTesting_env\lib\site-packages\gymnasium\spaces\box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
c:\Users\mariujf\anaconda3\envs\RLTesting_env\lib\site-packages\gymnasium\spaces\box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
